### Decorators

a design pattern in Python that allows a user to add new functionality to an existing object without modifying its structure. 

In [84]:
def loop_through(lst):
    for element in lst:
        print(element)

In [85]:
lst = [None]*10
loop_through(lst)

None
None
None
None
None
None
None
None
None
None


Suppose I have some list - "lst" - that I want to iterate through like the above. What if I want to time how long it takes for the loop through all the elements in the list, depending on its size? I could do something like the below.

In [86]:
from time import perf_counter

In [87]:
def loop_through(size=1000):
    lst = [None]*size
    now = perf_counter()
    for element in lst:
        pass
    total_time = perf_counter() - now
    print(f'ran in: {total_time}')

In [88]:
loop_through(1000)

ran in: 1.6807000065455213e-05


but we may have other useful functions we want to time, and we may want to run these mutliple times to get an average run time or an aggregate that is more comprehensible. A decorator should do the trick. 

In [81]:
def timeit(func):
    def wrapper(*args, **kwargs):
        now = perf_counter()
        res = func(*args, **kwargs)
        total_time = perf_counter() - now
        print(f'{func.__name__} ran in: {total_time}')
        return res
    return wrapper

In [89]:
@timeit
def new_loop_through(size=1000):
    lst = [None]*size
    for element in lst:
        pass    

In [90]:
new_loop_through()

new_loop_through ran in: 2.478399983374402e-05


In [91]:
@timeit
def sum_first_n_brute_force(n=1000):
    return sum([i for i in range(1, n+1)])

@timeit
def sum_first_n_formula(n=1000):
    return n*(n+1)/2

In [92]:
print(sum_first_n_brute_force())
print(sum_first_n_formula())

sum_first_n_brute_force ran in: 9.132900004260591e-05
500500
sum_first_n_formula ran in: 2.3849997887737118e-06
500500.0


adding the @timeit above a function is equivalent to the below

In [45]:
def my_func():
    pass

my_func = timeit(my_func)

In [46]:
my_func()

my_func ran in: 0:00:00.000009


so what if we want to take the aggregate of a number of runs? We would need our timeit function to take some arguments. We might want something like the below.

In [ ]:
@timeit(iters=10000)
def my_func():
    pass

If we look at the expressions above, we see that this would be equivalent to

In [ ]:
def my_func():
    pass

my_func = timeit(iters=10000)(my_func)

This means we essentially need our original decorator back after calling the timeit function with the iters argument. So we need to do something like below

In [93]:
def timeit(iters=10000):
    def inner(func):
        def wrapper(*args, **kwargs):
            total_time = 0
            for i in range(iters):
                now = perf_counter()
                res = func(*args, **kwargs)
                total_time += perf_counter() - now
            print(f'{iters} iterations of {func.__name__} ran in: {total_time}')
            return res
        return wrapper
    return inner

In [94]:
@timeit(iters=15000)
def my_func():
    pass
my_func()

15000 iterations of my_func ran in: 0.0033480139013590815


In [95]:
@timeit(iters=20000)
def sum_first_n_formula(n=1000):
    return n*(n+1)/2

In [96]:
sum_first_n_formula()

20000 iterations of sum_first_n_formula ran in: 0.006994030004989327


500500.0

Have to call timeit as a function if we pass no args, way to take care of this but will avoid for now as we have a solution later

In [97]:
@timeit()
def my_func_2(n=1):
    return n+1

my_func_2()

10000 iterations of my_func_2 ran in: 0.0022722830021848495


2

In [98]:
@timeit
def my_func_2(n=1):
    return n+1

my_func_2()

TypeError: inner() missing 1 required positional argument: 'func'

#### What about decorators on class instance methods?

In [99]:
def reduce_repeated_class_logic(func):
    def wrapper(self, *args, **kwargs):
        self.is_important = not self.is_important
        return func(self, *args, **kwargs)
    return wrapper

class MyClass:
    is_important: bool=True
    
    @reduce_repeated_class_logic
    def do_something_important(self):
        pass
    
    @reduce_repeated_class_logic
    def do_another_important_thing_similar_to_above(self):
        pass
    

In [100]:
my_class = MyClass()
my_class.is_important

True

In [101]:
my_class.do_something_important()
my_class.is_important

False

In [102]:
my_class.do_another_important_thing_similar_to_above()
my_class.is_important

True

Decorator module to help with writing decorators

In [103]:
from decorator import decorator

In [104]:
@decorator
def reduce_repeated_class_logic_2(func, self, *args, **kwargs):
    self.is_important = not self.is_important
    return func(self, *args, **kwargs)

In [105]:
class MyClass:
    is_important: bool=True
    
    @reduce_repeated_class_logic_2
    def do_something_important(self):
        pass
    
    @reduce_repeated_class_logic_2
    def do_another_important_thing_similar_to_above(self):
        pass

In [106]:
my_class = MyClass()
my_class.is_important

True

In [107]:
my_class.do_something_important()
my_class.is_important

False

Decorators are not restricted to just functions, they can be used on classes as well. The below takes the concept above one step further by decorating all the class methods using a decorator on the class itself

In [110]:
def decorate_methods(decorator, exclude_list=None):
    exclude_list = [] if exclude_list is None else exclude_list
    def decorate_methods_inner(cls):
        for attr in cls.__dict__:
            if callable(getattr(cls, attr)) and '__' not in attr and attr not in exclude_list:
                setattr(cls, attr, decorator(getattr(cls, attr)))
        return cls
    return decorate_methods_inner

In [111]:
@decorate_methods(reduce_repeated_class_logic_2, exclude_list=['exclude'])
class MyClass:
    is_important: bool=True
    
    def do_something_important(self):
        pass
    
    def do_another_important_thing_similar_to_above(self):
        pass
    
    def exclude(self):
        pass

In [112]:
my_class = MyClass()
print(my_class.is_important)
my_class.do_something_important()
print(my_class.is_important)
my_class.do_another_important_thing_similar_to_above()
print(my_class.is_important)
my_class.exclude
print(my_class.is_important)

True
False
True
True
